#Import library

In [4]:
from google.colab import drive
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
from collections import Counter
from PIL import Image

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


#Loading 💡

In [6]:
folder_path = "/content/drive/MyDrive/Project_JOB_Analysis/data"

In [7]:
dataframe = []

In [8]:
for file_name in os.listdir(folder_path):
    if file_name.endswith(".csv"):
        file_path = os.path.join(folder_path, file_name)
        print(f"Now reading file: {file_name}")
        df = pd.read_csv(file_path, encoding='latin1')

        dataframe.append(df)

Now reading file: data_us.csv
Now reading file: data_aus.csv
Now reading file: data_vn.csv
Now reading file: data_us_2.csv
Now reading file: data_aus_2.csv
Now reading file: data_vn_2.csv


In [9]:
#Merges all data

merged_df = pd.concat(dataframe, ignore_index=True)

#merged_df.to_csv('/content/drive/MyDrive/Project_JOB_Analysis/merge_data.csv', index=False)

In [10]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1517 entries, 0 to 1516
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   applicationsCount  1516 non-null   object 
 1   applyType          1517 non-null   object 
 2   applyUrl           1516 non-null   object 
 3   benefits           84 non-null     object 
 4   companyId          1490 non-null   float64
 5   companyName        1490 non-null   object 
 6   companyUrl         1490 non-null   object 
 7   contractType       1516 non-null   object 
 8   description        1516 non-null   object 
 9   experienceLevel    1516 non-null   object 
 10  id                 1517 non-null   int64  
 11  jobUrl             1516 non-null   object 
 12  location           1516 non-null   object 
 13  postedTime         1516 non-null   object 
 14  posterFullName     405 non-null    object 
 15  posterProfileUrl   405 non-null    object 
 16  publishedAt        1517 

#Cleaning 👏

In [11]:
#List of relevant columns
relevant_columns = [
    "title",
    "experienceLevel",
    "salary",
    "benefits",
    "companyName",
    "contractType",
    "location",
    "sector",
    "workType",
    "description",
    "publishedAt",
    "id",
]

In [12]:
merged_df = merged_df[relevant_columns]

##Check dup

In [13]:
#Check duplicate

duplicates = merged_df.duplicated().sum()

In [14]:
print(duplicates)

306


In [15]:
merged_df = merged_df.drop_duplicates()

## Handle null

In [16]:
missing_values = merged_df.isna().sum()

missing_values

,0
title,1
experienceLevel,1
salary,978
benefits,1139
companyName,21
contractType,1
location,1
sector,27
workType,3
description,1


In [17]:
merged_df = merged_df.dropna(subset=['title'])
merged_df = merged_df.dropna(subset=['location'])

In [18]:
#Check null
print(merged_df['salary'].isnull().sum())

977


In [19]:
#Fill null val
merged_df['salary'] = merged_df['salary'].fillna('Not Disclosed')
merged_df['benefits'] = merged_df['benefits'].fillna('No Benefits')

#Preprocess

##Create country column for Location

In [20]:
merged_df['location'] = merged_df['location'].astype(str)

In [21]:
#Dictionary mapping
location_mapping = {
    #Aus States
    'New South Wales': 'Australia', 'Victoria': 'Australia', 'Queensland': 'Australia',
    'Western Australia': 'Australia', 'South Australia': 'Australia', 'Tasmania': 'Australia','Australia': 'Australia','Sydney': 'Australia', 'Melbourne': 'Australia', 'Perth': 'Australia',
    #Vietnamese Regions
    'Hanoi': 'Vietnam', 'Ho Chi Minh': 'Vietnam', 'Da Nang': 'Vietnam', 'Can Tho': 'Vietnam','Vietnam': 'Vietnam'
}

def categorize_location(location):
    location = location.lower()  # Normalize to lowercase for case-insensitive matching
    for key, country in location_mapping.items():
        if key.lower() in location:
            return country
    return 'US'

#Apply the function
merged_df['country'] = merged_df['location'].apply(categorize_location)

In [22]:
merged_df.drop(columns=['location'], inplace=True)

##Standarlize salary columns

In [23]:
AUD_TO_USD = 0.65
INR_TO_USD = 0.012
VND_TO_USD = 0.000041

In [24]:
def convert_to_usd(salary_str, country):
    if "Not Disclosed" in salary_str or not isinstance(salary_str, str):
        return "Not Disclosed"

    try:
        # Chuẩn hóa chuỗi lương
        salary_str = salary_str.strip()
        salary_str = salary_str.replace("?", "").replace("$", "").replace("A$", "").replace(",", "").replace("A", "").replace("M", "000000")

        if country == "US":
            # Xử lý USD
            if "/hr" in salary_str:
                ranges = [float(s.replace("/hr", "").strip()) for s in salary_str.split("-")]
                avg_hourly_rate = sum(ranges) / len(ranges)
                yearly_salary = avg_hourly_rate * 8 * 22 * 12  # Chuyển đổi sang lương năm
                return yearly_salary
            if "/yr" in salary_str:
                ranges = [float(s.replace("/yr", "").strip()) for s in salary_str.split("-")]
                return sum(ranges) / len(ranges)

        elif country == "Australia":
            # Xử lý AUD
            if "/hr" in salary_str:
                ranges = [float(s.replace("/hr", "").strip()) for s in salary_str.split("-")]
                avg_hourly_rate = sum(ranges) / len(ranges)
                yearly_salary = avg_hourly_rate * 8 * 22 * 12  # Chuyển đổi sang lương năm
                return yearly_salary * AUD_TO_USD
            if "/daily" in salary_str:
                ranges = [float(s.replace("/daily", "").strip()) for s in salary_str.split("-")]
                avg_daily_rate = sum(ranges) / len(ranges)
                yearly_salary = avg_daily_rate * 22 * 12  # Chuyển đổi sang lương năm
                return yearly_salary * AUD_TO_USD
            if "/yr" in salary_str:
                ranges = [float(s.replace("/yr", "").strip()) for s in salary_str.split("-")]
                avg_salary = sum(ranges) / len(ranges)
                return avg_salary * AUD_TO_USD

        elif country == "Vietnam":
            # Xử lý VND hoặc INR
            if "/hr" in salary_str:
                ranges = [float(s.replace("/hr", "").strip()) for s in salary_str.split("-")]
                avg_hourly_rate = sum(ranges) / len(ranges)
                yearly_salary = avg_hourly_rate * 8 * 22 * 12
                return yearly_salary * VND_TO_USD
            if "/mo" in salary_str:
                ranges = [float(s.replace("/mo", "").strip()) for s in salary_str.split("-")]
                avg_monthly_salary = sum(ranges) / len(ranges)
                yearly_salary = avg_monthly_salary * 12
                return yearly_salary * VND_TO_USD
            if "/yr" in salary_str:
                ranges = [float(s.replace("/yr", "").replace("M", "000000").strip()) for s in salary_str.split("-")]
                avg_salary = sum(ranges) / len(ranges)
                return avg_salary * INR_TO_USD

    except Exception as e:
        # Ghi log lỗi
        print(f"Error processing salary: {salary_str}, country: {country}. Error: {e}")
        return np.nan


In [25]:
# Apply the conversion function
merged_df['normalized_salary_usd'] = merged_df.apply(
    lambda row: convert_to_usd(row['salary'], row['country']), axis=1
)

##Create skill col from "Description" columns


In [26]:
skills = [
    'python', 'sql', 'excel', 'r studio', 'power bi', 'powerbi', 'tableau', 'machine learning',
    'deep learning', 'aws', 'azure', 'gcp', 'hadoop', 'spark', 'data visualization',
    'data analytics', 'statistics', 'numpy', 'pandas', 'scikit-learn', 'nlp', 'big data',

    'google analytics', 'data warehouse', 'snowflake', 'brick', 'mysql', 'sql server', 'postgres', 'olap', 'dax', 'power platform', 'power automate', 'database'
]

In [27]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text


merged_df['clean_description'] = merged_df['description'].apply(preprocess_text)

In [28]:
def extract_skills(text, skills_list):
    found_skills = [skill for skill in skills_list if skill in text]
    return found_skills

merged_df['skills'] = merged_df['clean_description'].apply(lambda x: extract_skills(x, skills))

In [29]:
all_skills =merged_df['skills'].sum()
skill_counts = Counter(all_skills)


skill_df = pd.DataFrame(skill_counts.items(), columns=['Skill', 'Frequency'])
skill_df = skill_df.sort_values(by='Frequency', ascending=False)

#print(skill_df.head(20))

In [30]:
#Dedup if one job mention a skill twice or more
def extract_unique_skills(text, skills_list):
    text = text.replace('powerbi', 'power bi')
    found_skills = {skill for skill in skills_list if skill in text}  # Use a set for unique values
    return list(found_skills)  # Convert back to list if needed

merged_df['skills'] = merged_df['clean_description'].apply(lambda x: extract_unique_skills(x, skills))

In [31]:
all_unique_skills = merged_df['skills'].sum()

skill_counts = Counter(all_unique_skills)

skill_df = pd.DataFrame(skill_counts.items(), columns=['Skill', 'Frequency'])
skill_df = skill_df.sort_values(by='Frequency', ascending=False)

#print(skill_df.head(20))

# Checking

In [32]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=merged_df)

https://docs.google.com/spreadsheets/d/1HQdrUxQK4Idik6ugEsLLX5vYPyzdzWhSRj1whytJg6U#gid=0


In [33]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1210 entries, 0 to 1516
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   title                  1210 non-null   object
 1   experienceLevel        1210 non-null   object
 2   salary                 1210 non-null   object
 3   benefits               1210 non-null   object
 4   companyName            1190 non-null   object
 5   contractType           1210 non-null   object
 6   sector                 1184 non-null   object
 7   workType               1208 non-null   object
 8   description            1210 non-null   object
 9   publishedAt            1210 non-null   object
 10  id                     1210 non-null   int64 
 11  country                1210 non-null   object
 12  normalized_salary_usd  1210 non-null   object
 13  clean_description      1210 non-null   object
 14  skills                 1210 non-null   object
dtypes: int64(1), object(14)
me

# EDA

## Base on experience level and country

In [34]:
#Count of experience levels
experience_level_counts = merged_df['experienceLevel'].value_counts().reset_index()
experience_level_counts.columns = ['experienceLevel', 'count']

#Bar plot
fig = px.bar(
    experience_level_counts,
    x='experienceLevel',
    y='count',
    title='Distribution of Experience Level',
    labels={'experienceLevel': 'Experience Level', 'count': 'Count'},
    color='count',
    color_continuous_scale='Blues'
)

#Custom layout
fig.update_layout(
    template='plotly_dark',
    xaxis_title='Experience Level',
    yaxis_title='Count',
    font=dict(size=15, family="Franklin Gothic"),
    height=500
)

fig.show()


In [35]:
#Counts of countries
country_counts = merged_df['country'].value_counts().reset_index()
country_counts.columns = ['country', 'count']

#Bar plot
fig = px.bar(
    country_counts,
    x='country',
    y='count',
    title='Distribution of Country',
    labels={'country': 'Country', 'count': 'Count'},
    color='count',
    color_continuous_scale='Blues'
)

#Custom layout
fig.update_layout(
    template='plotly_dark',
    xaxis_title='Country',
    yaxis_title='Count',
    font=dict(size=15, family="Franklin Gothic"),
    height=500
)

fig.show()

In [36]:
# Count occurr of experience levels grouped by country
experience_by_country = merged_df.groupby(['experienceLevel', 'country']).size().reset_index(name='count')

#Bar plot
fig = px.bar(
    experience_by_country,
    x='experienceLevel',
    y='count',
    color='country',
    title='Experience Level by Country',
    labels={'experienceLevel': 'Experience Level', 'count': 'Count', 'country': 'Country'},
    barmode='group',
    color_discrete_sequence=px.colors.sequential.Blues
)

#Add annotations for counts
for i, row in experience_by_country.iterrows():
    fig.add_annotation(
        x=row['experienceLevel'],
        y=row['count'],
        text=str(row['count']),
        showarrow=False,
        font=dict(color="white", size=12)
    )

#=Custom layout
fig.update_layout(
    template='plotly_dark',
    xaxis_title='Experience Level',
    yaxis_title='Count',
    font=dict(size=15, family="Franklin Gothic"),
    height=600
)

fig.show()



In [37]:
# Filter out rows where experienceLevel is "Not applicable"
filtered_df = merged_df[merged_df['experienceLevel'] != 'Not applicable']

# Count occurrences of experience levels grouped by country
experience_by_country = filtered_df.groupby(['experienceLevel', 'country']).size().reset_index(name='count')

# Bar plot
fig = px.bar(
    experience_by_country,
    x='experienceLevel',
    y='count',
    color='country',
    title='Experience Level by Country',
    labels={'experienceLevel': 'Experience Level', 'count': 'Count', 'country': 'Country'},
    barmode='group',
    color_discrete_sequence=px.colors.sequential.Blues
)

# Add annotations for counts
for i, row in experience_by_country.iterrows():
    fig.add_annotation(
        x=row['experienceLevel'],
        y=row['count'],
        text=str(row['count']),
        showarrow=False,
        font=dict(color="white", size=12)
    )

# Custom layout
fig.update_layout(
    template='plotly_dark',
    xaxis_title='Experience Level',
    yaxis_title='Count',
    font=dict(size=15, family="Franklin Gothic"),
    height=600
)

fig.show()


In [38]:
experience_by_country

,experienceLevel,country,count
0,Associate,Australia,45
1,Associate,US,100
2,Associate,Vietnam,22
3,Director,Australia,1
4,Director,US,1
5,Director,Vietnam,4
6,Entry level,Australia,87
7,Entry level,US,120
8,Entry level,Vietnam,131
9,Executive,US,1


In [39]:
import plotly.express as px
import pandas as pd

# Filter out rows where experienceLevel is "Not applicable"
filtered_df = merged_df[merged_df['experienceLevel'] != 'Not applicable']

# Count occurrences of experience levels grouped by country
experience_by_country = filtered_df.groupby(['experienceLevel', 'country']).size().reset_index(name='count')

# Create the bar plot
fig = px.bar(
    experience_by_country,
    x='experienceLevel',
    y='count',
    color='country',
    title='Experience Level by Country',
    labels={'experienceLevel': 'Experience Level', 'count': 'Count', 'country': 'Country'},
    barmode='group',
    color_discrete_sequence=px.colors.sequential.Blues
)

# Add annotations for counts with proper positioning
for i, row in experience_by_country.iterrows():
    # Determine the x-position based on the bar's grouping
    x_position = row['experienceLevel']
    y_position = row['count'] + 0.5  # Slightly above the bar

    # Add the annotation for the count value
    fig.add_annotation(
        x=x_position,
        y=y_position,
        text=str(row['count']),
        showarrow=False,
        font=dict(color="white", size=12),
        align="center"
    )

# Custom layout
fig.update_layout(
    template='plotly_dark',
    xaxis_title='Experience Level',
    yaxis_title='Count',
    font=dict(size=15, family="Franklin Gothic"),
    height=600
)

fig.show()



## Base on Salary

In [40]:
grouped_by_exp = merged_df.groupby('experienceLevel')['normalized_salary_usd'].describe()
print(grouped_by_exp)

                 count unique            top freq
experienceLevel                                  
Associate          167     29  Not Disclosed  137
Director             6      2  Not Disclosed    5
Entry level        338     38  Not Disclosed  292
Executive            4      1  Not Disclosed    4
Internship          31      7  Not Disclosed   24
Mid-Senior level   556     84  Not Disclosed  437
Not Applicable     108     24  Not Disclosed   78


In [41]:
data_cleaned = merged_df[merged_df['normalized_salary_usd'] != 'Not Disclosed']
data_cleaned['normalized_salary_usd'] = pd.to_numeric(data_cleaned['normalized_salary_usd'], errors='coerce')

<ipython-input-41-6915e2e21c4f>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
#Create df top 10 highest salaries
top_10_highest_salaries = data_cleaned.nlargest(10, 'normalized_salary_usd')
top_10_df = top_10_highest_salaries[['title', 'normalized_salary_usd', 'country']]
top_10_df['label'] = top_10_df['title'] + " (" + top_10_df['country'] + ")"

#Bar plot
fig = px.bar(
    top_10_df,
    x='normalized_salary_usd',
    y='label',
    orientation='h',
    title='Top 10 Highest Salaries by Job Title and Country',
    labels={'normalized_salary_usd': 'Salary (USD)', 'label': 'Job Title (Country)'},
    color='normalized_salary_usd',
    color_continuous_scale='Blues'
)

#Customlayout
fig.update_layout(
    template='plotly_dark',
    xaxis_title='Salary (USD)',
    yaxis_title='Job Title (Country)',
    font=dict(size=15, family="Franklin Gothic"),
    height=600
)

# Show the figure
fig.show()




<ipython-input-42-115008cb959b>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [43]:
top_10_df

,title,normalized_salary_usd,country,label
634,Data Scientist (L4) - Content DSE,445000.0,US,Data Scientist (L4) - Content DSE (US)
1129,Python Engineer - Risk & PnL,390000.0,Australia,Python Engineer - Risk & PnL (Australia)
758,Data Analyst,300000.0,US,Data Analyst (US)
89,"Staff Analyst, Advanced Analytics",207750.0,US,"Staff Analyst, Advanced Analytics (US)"
130,"Staff Analyst, Growth Analytics",187250.0,US,"Staff Analyst, Growth Analytics (US)"
685,"Staff Analyst, Growth Analytics",187250.0,US,"Staff Analyst, Growth Analytics (US)"
15,Business Data Analyst,185000.0,US,Business Data Analyst (US)
171,Data Analyst,171600.0,Australia,Data Analyst (Australia)
290,Senior Data Analyst // Data Privacy or Records...,171600.0,Australia,Senior Data Analyst // Data Privacy or Records...
1198,Data Engineer,171600.0,Australia,Data Engineer (Australia)


In [44]:
#average salary by country
average_salary_by_country = data_cleaned.groupby('country')['normalized_salary_usd'].mean().sort_values(ascending=False).reset_index()

#Bar plot
fig = px.bar(
    average_salary_by_country,
    x='country',
    y='normalized_salary_usd',
    title='Average Salary by Country',
    labels={'normalized_salary_usd': 'Average Salary (USD)', 'country': 'Country'},
    color='normalized_salary_usd',
    color_continuous_scale='Blues'
)

# Customize layout
fig.update_layout(
    template='plotly_dark',
    xaxis_title='Country',
    yaxis_title='Average Salary (USD)',
    font=dict(size=15, family="Franklin Gothic"),
    height=600
)

fig.show()

In [45]:
average_salary_by_country

,country,normalized_salary_usd
0,US,109351.878483
1,Australia,106967.741679
2,Vietnam,5331.510956


## Base on sector

In [46]:
# Group data by country and sector and count the number of rows (jobs) in each sector
sector_analysis = data_cleaned.groupby(['country', 'sector']).size().reset_index(name='job_count')

# Sort by job count in descending order within each country
sector_analysis_sorted = sector_analysis.sort_values(by=['country', 'job_count'], ascending=[True, False])

# Select top 5 sectors with the highest job count for each country
top_sectors_by_country = sector_analysis_sorted.groupby('country').head(5).reset_index(drop=True)

# Bar plot
fig = px.bar(
    top_sectors_by_country,
    x='country',
    y='job_count',
    color='sector',
    title='Top 5 Sectors by Job Count per Country',
    labels={'job_count': 'Job Count', 'country': 'Country'},
    color_discrete_sequence=px.colors.sequential.Blues
)

# Custom layout
fig.update_layout(
    template='plotly_dark',
    xaxis_title='Country',
    yaxis_title='Job Count',
    font=dict(size=15, family="Franklin Gothic"),
    height=600
)

In [47]:
top_sectors_by_country

,country,sector,job_count
0,Australia,Financial Services,5
1,Australia,Data Infrastructure and Analytics,3
2,Australia,IT Services and IT Consulting,3
3,Australia,Software Development,3
4,Australia,Staffing and Recruiting,3
5,US,Software Development,23
6,US,Financial Services,12
7,US,Hospitals and Health Care,11
8,US,"Technology, Information and Internet",10
9,US,IT Services and IT Consulting,9


In [48]:
#Group data by country, sector, and job title
sector_analysis = data_cleaned.groupby(['country', 'sector'])['normalized_salary_usd'].mean().reset_index()

#Sort by salary in descending order within each country
sector_analysis_sorted = sector_analysis.sort_values(by=['country', 'normalized_salary_usd'], ascending=[True, False])

#Select top 5 sectors with the highest salary for each country
top_sectors_by_country = sector_analysis_sorted.groupby('country').head(5).reset_index(drop=True)

# Bar plot
fig = px.bar(
    top_sectors_by_country,
    x='country',
    y='normalized_salary_usd',
    color='sector',
    title='Top 5 High-Paying Sectors by Country',
    labels={'normalized_salary_usd': 'Average Salary (USD)', 'country': 'Country'},
    color_discrete_sequence=px.colors.sequential.Blues
)

#Custom layout
fig.update_layout(
    template='plotly_dark',
    xaxis_title='Country',
    yaxis_title='Average Salary (USD)',
    font=dict(size=15, family="Franklin Gothic"),
    height=600
)


In [49]:
top_sectors_by_country

,country,sector,normalized_salary_usd
0,Australia,"Capital Markets, Software Development, and Tec...",390000.000000
1,Australia,"Banking, Investment Banking, and IT Services a...",171600.000000
2,Australia,"Education, Government Administration, and Educ...",144144.000000
3,Australia,"Health and Human Services, Information Service...",144144.000000
4,Australia,Information Technology & Services,137280.000000
5,US,Entertainment Providers,197977.750000
6,US,Banking,155000.000000
7,US,Public Health,154902.500000
8,US,Consumer Electronics,148500.000000
9,US,Electric Power Generation,142560.000000


##Base on skills

In [50]:
import base64

# Function to find and encode logo as base64
def encode_logo(skill, logos_folder='logos'):
    """Find and encode a matching logo file for a given skill."""
    extensions = ['.png', '.jpg', '.jpeg', '.svg', '.gif']
    skill_cleaned = skill.lower().replace(' ', '')

    for ext in extensions:
        filepath = os.path.join(logos_folder, f"{skill_cleaned}{ext}")
        if os.path.exists(filepath):
            with open(filepath, "rb") as image_file:
                return f"data:image/{ext.lstrip('.')};base64," + base64.b64encode(image_file.read()).decode()

    return None

# Sample data
df = skill_df.copy()

# Add logos as base64 images
df['Logo'] = df['Skill'].apply(lambda skill: encode_logo(skill) or '')

# Sort DataFrame by Frequency
df = df.sort_values('Frequency', ascending=False)

# Bar plot with logos
fig = px.bar(
    df,
    x='Frequency',
    y='Skill',
    orientation='h',
    title='Skills Frequency Distribution',
    labels={'Frequency': 'Count', 'Skill': 'Skills'},
    text='Frequency',
    color='Frequency',
    color_continuous_scale='Blues'
)

# Customize layout
fig.update_layout(
    template='plotly_dark',
    xaxis_title='Count',
    yaxis_title='',
    font=dict(size=15, family="Franklin Gothic"),
    height=800
)

# Add logos to the y-axis ticks
fig.update_yaxes(
    tickmode='array',
    tickvals=list(range(len(df))),
    ticktext=[
        f'<img src="{logo}" style="height:20px; margin-right:10px;"> {skill}' if logo else skill
        for skill, logo in zip(df['Skill'], df['Logo'])
    ],
    title_standoff=25
)

# Show the figure
fig.show()

In [51]:
# Function to find and encode logo as base64
def encode_logo(skill, logos_folder='logos'):
    """Find and encode a matching logo file for a given skill."""
    extensions = ['.png', '.jpg', '.jpeg', '.svg', '.gif']
    skill_cleaned = skill.lower().replace(' ', '')

    for ext in extensions:
        filepath = os.path.join(logos_folder, f"{skill_cleaned}{ext}")
        if os.path.exists(filepath):
            with open(filepath, "rb") as image_file:
                return f"data:image/{ext.lstrip('.')};base64," + base64.b64encode(image_file.read()).decode()

    return None

# Sample data
df = skill_df.copy()

# Add logos as base64 images
df['Logo'] = df['Skill'].apply(lambda skill: encode_logo(skill) or '')

# Sort DataFrame by Frequency in descending order
df = df.sort_values('Frequency', ascending=True)  # Change to ascending=True to place the most required skill at the top

# Bar plot with logos
fig = px.bar(
    df,
    x='Frequency',
    y='Skill',
    orientation='h',
    title='Skills Frequency Distribution',
    labels={'Frequency': 'Count', 'Skill': 'Skills'},
    text='Frequency',
    color='Frequency',
    color_continuous_scale='Blues'
)

# Customize layout
fig.update_layout(
    template='plotly_dark',
    xaxis_title='Count',
    yaxis_title='',
    font=dict(size=15, family="Franklin Gothic"),
    height=800
)

# Add logos to the y-axis ticks
fig.update_yaxes(
    tickmode='array',
    tickvals=list(range(len(df))),
    ticktext=[
        f'<img src="{logo}" style="height:20px; margin-right:10px;"> {skill}' if logo else skill
        for skill, logo in zip(df['Skill'], df['Logo'])
    ],
    title_standoff=25
)

# Show the figure
fig.show()

## Base on job post date

In [52]:
# Convert 'publishedAt' to datetime
merged_df['publishedAt'] = pd.to_datetime(merged_df['publishedAt'], format='%Y-%m-%d')

# Extract the month and year from 'publishedAt'
merged_df['Month'] = merged_df['publishedAt'].dt.month
merged_df['Year'] = merged_df['publishedAt'].dt.year

# Group by Year and Month to count the number of jobs
jobs_by_month = merged_df.groupby(['Year', 'Month'])['id'].count().reset_index()

# Rename columns for clarity
jobs_by_month.columns = ['Year', 'Month', 'JobCount']

print(jobs_by_month)


    Year  Month  JobCount
0   2023      8         1
1   2024      3         1
2   2024      4         2
3   2024      5         1
4   2024      6         5
5   2024      7        10
6   2024      8        24
7   2024      9        44
8   2024     10        61
9   2024     11       410
10  2024     12       651


In [53]:
# Extract week and year from 'publishedAt'
merged_df['Year'] = merged_df['publishedAt'].dt.year
merged_df['Week'] = merged_df['publishedAt'].dt.isocalendar().week

# Filter data for November and December
merged_df = merged_df[(merged_df['Year'] == 2024) & (merged_df['publishedAt'].dt.month.isin([11, 12]))]

In [54]:
# Group by country, week, and count the number of job postings
job_trends_weekly_country = merged_df.groupby(['country', 'Year', 'Week']).size().reset_index(name='JobCount')

# Normalize the weeks to start from 1 for November
job_trends_weekly_country['AdjustedWeek'] = job_trends_weekly_country['Week'] - job_trends_weekly_country['Week'].min() + 1


In [55]:
job_trends_weekly_country

,country,Year,Week,JobCount,AdjustedWeek
0,Australia,2024,44,2,1
1,Australia,2024,45,2,2
2,Australia,2024,46,24,3
3,Australia,2024,47,39,4
4,Australia,2024,48,79,5
5,Australia,2024,49,113,6
6,Australia,2024,50,121,7
7,US,2024,44,2,1
8,US,2024,45,6,2
9,US,2024,46,21,3


In [56]:
# Create a bar plot for the job trends by week and country
fig = px.line(
    job_trends_weekly_country,
    x='AdjustedWeek',
    y='JobCount',
    color='country',
    title='Job Postings Trends by Week for November and December 2024 by Country',
    labels={'JobCount': 'Number of Job Postings', 'AdjustedWeek': 'Week (November 1st = Week 1)'},
)

# Custom layout
fig.update_layout(
    template='plotly_dark',
    xaxis_title='Week (November 1st = Week 1)',
    yaxis_title='Number of Job Postings',
    font=dict(size=15, family="Franklin Gothic"),
    height=600
)

# Show the plot
fig.show()

#Download for next step

In [57]:
merged_df = merged_df.drop(columns=['description', 'clean_description'])

In [58]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1061 entries, 0 to 1516
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   title                  1061 non-null   object        
 1   experienceLevel        1061 non-null   object        
 2   salary                 1061 non-null   object        
 3   benefits               1061 non-null   object        
 4   companyName            1041 non-null   object        
 5   contractType           1061 non-null   object        
 6   sector                 1035 non-null   object        
 7   workType               1059 non-null   object        
 8   publishedAt            1061 non-null   datetime64[ns]
 9   id                     1061 non-null   int64         
 10  country                1061 non-null   object        
 11  normalized_salary_usd  1061 non-null   object        
 12  skills                 1061 non-null   object        
 13  Month   

In [59]:
merged_df.to_csv('/content/drive/MyDrive/Project_JOB_Analysis/merge_data2.csv', index=False)